# C

In [1]:
#!pip install tensorflow==2.12


In [2]:
import tensorflow as tf

In [3]:
#!pip install numpy==1.23
#!pip install matplotlib==3.9.4

In [4]:
#!pip install opencv-python

In [5]:
import os
import cv2
import numpy as np

from keras.utils import to_categorical 
from keras.models import Sequential 
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

In [6]:
#!pip list

In [7]:
dataset_path = "dataset/"

In [8]:
#image parameters
img_size = 128

In [9]:
#prepare data generator
datagen = ImageDataGenerator(rescale=1./255,
                             validation_split=0.2
                            )
train_data = datagen.flow_from_directory(
    dataset_path,
    target_size=(img_size, img_size),
    batch_size=32,
    class_mode="binary",
    subset="training"
)

val_data = datagen.flow_from_directory(
    dataset_path,
    target_size=(img_size, img_size),
    batch_size=32,
    class_mode="binary",
    subset="validation"
)

Found 3077 images belonging to 2 classes.
Found 769 images belonging to 2 classes.


In [10]:
#Build CNN Mode1
#Define image size(example: 128x128)
img_size = 128
model = Sequential([
    Conv2D(32, (3,3), activation="relu", input_shape=(img_size, img_size, 3)),
    MaxPooling2D(pool_size=(2,2)),
    
    Conv2D(64, (3,3), activation="relu"),
    MaxPooling2D(pool_size=(2,2)),
    
    Conv2D(128, (3,3), activation="relu"),
    MaxPooling2D(pool_size=(2,2)),

    Flatten(),
    Dense(128, activation="relu"),
    Dropout(0.5),
    Dense(1, activation="sigmoid")     #For binary classification
])

model.compile(loss="binary_crossentropy",
              optimizer=Adam(0.0001),
              metrics=["accuracy"]
             )

model.compile(
    loss="binary_crossentropy",
    optimizer=Adam(0.0001),
    metrics=["accuracy"]
    )

In [11]:
#Train Model
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=1
)

21/97 [=====>........................] - ETA: 38s - loss: 0.5966 - accuracy: 0.7158

C:\Users\Admin\anaconda3\envs\aidl\lib\site-packages\PIL\Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


97/97 [==============================] - 52s 534ms/step - loss: 0.3825 - accuracy: 0.8453 - val_loss: 0.1868 - val_accuracy: 0.9428


In [12]:
#save model
model.save("face_mask_cnn.h5")
print("Model saved as face_mask_cnn.h5")

Model saved as face_mask_cnn.h5


In [13]:
#Real-Time Mask Detection
import cv2
import numpy as np
from tensorflow.keras.models import load_model
model = load_model("face_mask_cnn.h5")
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()
    if not ret or frame is None:
        break  #or continue,depending on your logic
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)
    for (x, y, w, h) in faces:
        face = frame[y:y+h, x:x+w]
        face =cv2.resize(face, (128, 128))
        face = face.astype("float32") / 235
        face = np.expand_dims(face, axis=0)
        prediction = model.predict(face)[0][0]
        label = "mask" if prediction < 0.5 else "No Mask"
        color = (0, 255, 0) if prediction < 0.5 else (0, 0, 255)
        cv2.putText(frame, label, (x, y-10), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)
        cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
        cv2.imshow("Mask Detection", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 32ms/step
